In [ ]:
# 머리에 딸기 장식이 있고 휘핑크림처럼 생긴 포켓몬
# 솜사탕처럼 생겼고 분홍색임
# 피카츄를 흉내낸 포켓몬
# 둥글둥글한 고슴도치 포켓몬
# 초록 갈색 깃털에 잎사귀 리본을 매고 있는 둥근 올빼미 같은 포켓몬
# 꼬리 끝에 불꽃이 있는 주황색 도마뱀 같은 포켓몬
# 보라색 풍선처럼 생긴 포켓몬

In [ ]:
user_query = "노란색 전기 뚱땡이"  # 예시 

# (1) SBERT 임베딩
sbert_query_emb = sbert_model.encode(user_query)
sbert_query_emb = sbert_query_emb / np.linalg.norm(sbert_query_emb)

# (2) CLIP 임베딩
clip_inputs = clip_processor(text=[user_query], return_tensors="pt", padding=True).to(clip_model.device)
with torch.no_grad():
    clip_query_emb = clip_model.get_text_features(**clip_inputs)
    clip_query_emb = clip_query_emb.cpu().numpy().flatten()
clip_query_emb = clip_query_emb / np.linalg.norm(clip_query_emb)

k = 50  # 후보군 크기, 실험적으로 조정 가능

# (1) SBERT top-k
_, sbert_top_idx = sbert_index.search(sbert_query_emb[np.newaxis, :].astype(np.float32), k)
# (2) CLIP top-k
_, clip_top_idx  = clip_index.search(clip_query_emb[np.newaxis, :].astype(np.float32), k)

# 후보군 id (합집합)
candidate_idx = set(sbert_top_idx[0]) | set(clip_top_idx[0])
candidate_ids = [id_list[idx] for idx in candidate_idx]

alpha, beta = 0.3, 0.7  # 가중치 (실험적으로 조정)
final_scores = {}

for pid in candidate_ids:
    # SBERT 점수 (코사인 유사도)
    sbert_score = np.dot(sbert_query_emb, sbert_text_embeddings[pid]) / (np.linalg.norm(sbert_query_emb) * np.linalg.norm(sbert_text_embeddings[pid]))
    # CLIP 점수 (코사인 유사도)
    clip_score  = np.dot(clip_query_emb, clip_img_embeddings[pid]) / (np.linalg.norm(clip_query_emb) * np.linalg.norm(clip_img_embeddings[pid]))
    # 결합 점수
    final_scores[pid] = alpha * sbert_score + beta * clip_score

# Top-N 결과 출력
topn = 5
topn_results = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:topn]

print(f"\n[사용자 입력]: {user_query}\n")
for rank, (pid, score) in enumerate(topn_results, 1):
    info = id2meta[pid]
    print(f"{rank}위: {info['name']} ({pid}) | 점수: {score:.4f}")
    print(f"설명: {info.get('full_description_ko','')}\n")
    # (이미지 경로: /kaggle/input/aiku-pokemon/pokemonimage/{pid}.png)


In [ ]:
from IPython.display import display, Image as IPyImage
import os

topn = 5  # top 5로 출력

print(f"\n[사용자 입력]: {user_query}\n")
for rank, (pid, score) in enumerate(topn_results[:topn], 1):
    info = id2meta[pid]
    img_path = f"/kaggle/input/aiku-pokemon/pokemonimage/{pid}.png"  # Kaggle 기준, Colab이면 경로만 바꿔주면 됨

    print(f"{rank}위: {info['name']} ({pid}) | 점수: {score:.4f}")
    display(IPyImage(filename=img_path))  # 이미지 띄우기
    print("-" * 40)
